# League of Legends Item Balancing
### Capstone Project 1: Data Wrangling

This documents the process I used to acquire the data for this project.

Some data will be acquired from the Riot Games Static API: https://ddragonexplorer.com/cdn/. This doesn't require a login.

The rest will be acquired from the Riot Games API: https://developer.riotgames.com/api-methods/. This does require a login. If you have an account for League of Legends, that will work.

In [4]:
import requests
import json
import math
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import time

In [5]:
# Do not store the API Key in a publicly available document :)
RIOT_API_KEY = 'RGAPI-09384eca-b5bd-4cbd-ba09-a46e2bdf76f0'

In [6]:
params = {'api_key': RIOT_API_KEY}

### TEST DATA PIPELINE

I walked through the whole process of acquiring match data with one data point, so I could troubleshoot and develop the method more quickly.

The process goes from league IDs to summoner IDs to account IDs to match histories to matches themselves.

Example Request to get an account ID from a summoner ID

In [8]:
account_request_link = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + 'F2deh05bxktXmEGhSR1Onfa-mIL_lmHYatpKnLZGGEE-5qo'
account = requests.get(account_request_link, params=params)
accounts_df = json_normalize(account.json())
accounts_df

,accountId,id,name,profileIconId,puuid,revisionDate,summonerLevel
0,H2UztO5Jzus8wtCEm-4x3xy-s0B3a24-Y8O8qHmPAHOCb1...,F2deh05bxktXmEGhSR1Onfa-mIL_lmHYatpKnLZGGEE-5qo,Zen Sano,770,txkJECCheRbC-lI41l0bZxkFnKGTvr6RSNUGa-6eY521sJ...,1550494712000,76


Example Request to get a match from a game ID. Game IDs can be found in match histories.

In [9]:
match_history_request_link ='https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/'+'H2UztO5Jzus8wtCEm-4x3xy-s0B3a24-Y8O8qHmPAHOCb1CcJxYtxjz2'+'?queue=420'
match_history = requests.get(match_history_request_link, params=params)
json_normalize(match_history.json()['matches'][0])

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,517,2980510552,TOP,NA1,420,SOLO,13,1550492534969


In [10]:
match_data_request_link = 'https://na1.api.riotgames.com/lol/match/v4/matches/' + str(int(2923933470L))
match_data = requests.get(match_data_request_link, params=params)
json_normalize(match_data.json(), record_path=['participants'], meta=['gameDuration'])

,championId,highestAchievedSeasonTier,participantId,spell1Id,spell2Id,stats,teamId,timeline,gameDuration
0,497,GOLD,1,4,3,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",100,"{u'lane': u'NONE', u'participantId': 1, u'csDi...",1191
1,20,PLATINUM,2,4,11,"{u'neutralMinionsKilledTeamJungle': 40, u'visi...",100,"{u'lane': u'NONE', u'participantId': 2, u'csDi...",1191
2,7,GOLD,3,4,14,"{u'neutralMinionsKilledTeamJungle': 8, u'visio...",100,"{u'lane': u'NONE', u'participantId': 3, u'csDi...",1191
3,202,PLATINUM,4,7,4,"{u'neutralMinionsKilledTeamJungle': 4, u'visio...",100,"{u'lane': u'NONE', u'participantId': 4, u'csDi...",1191
4,92,PLATINUM,5,14,4,"{u'neutralMinionsKilledTeamJungle': 4, u'visio...",100,"{u'lane': u'NONE', u'participantId': 5, u'csDi...",1191
5,236,PLATINUM,6,7,4,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",200,"{u'lane': u'NONE', u'participantId': 6, u'csDi...",1191
6,412,GOLD,7,14,4,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",200,"{u'lane': u'NONE', u'participantId': 7, u'csDi...",1191
7,86,UNRANKED,8,4,14,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",200,"{u'lane': u'NONE', u'participantId': 8, u'csDi...",1191
8,164,UNRANKED,9,4,11,"{u'neutralMinionsKilledTeamJungle': 55, u'visi...",200,"{u'lane': u'NONE', u'participantId': 9, u'csDi...",1191
9,38,UNRANKED,10,4,12,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",200,"{u'lane': u'NONE', u'participantId': 10, u'csD...",1191


Test Request. This request path gets the overall match information, for a given match id. While this has the set of items each player bought, the main information this may be missing is when each item was bought. This may be an important variable, and would require another request to find.

Using json_normalize took me a while to figure out.

Here is how not to use json_normalize:

In [11]:
match = json_normalize(match_data.json(), record_path=['participants'], meta=['gameId'])
match.head(2)

,championId,highestAchievedSeasonTier,participantId,spell1Id,spell2Id,stats,teamId,timeline,gameId
0,497,GOLD,1,4,3,"{u'neutralMinionsKilledTeamJungle': 0, u'visio...",100,"{u'lane': u'NONE', u'participantId': 1, u'csDi...",2923933470
1,20,PLATINUM,2,4,11,"{u'neutralMinionsKilledTeamJungle': 40, u'visi...",100,"{u'lane': u'NONE', u'participantId': 2, u'csDi...",2923933470


Most of the information I want is inside the stats column: I want that flattened.

The record_path / meta parameters think that you are leading them to rows. The entries should actually be columns. If I need multiple subsets of the full json, I will need to join them together. This is the correct way to use json_normalize:

In [12]:
match_stats = json_normalize(match_data.json()['participants'])
match_stats[match_stats['participantId'] == 1][['stats.item0', 'stats.item1',
                                                'stats.item2', 'stats.item'+'3',
                                                'stats.item4', 'stats.item5', 'stats.item6']]

,stats.item0,stats.item1,stats.item2,stats.item3,stats.item4,stats.item5,stats.item6
0,3098,3174,0,3117,0,0,3364


Above is a set of items at the end of a game. 4 of the 7 inventory slots have something in them, and 3 have nothing.

Below is an example of the full match data. There is a lot to work with, more than I will probably use.

In [13]:
match_stats

,championId,highestAchievedSeasonTier,participantId,spell1Id,spell2Id,stats.assists,stats.champLevel,stats.combatPlayerScore,stats.damageDealtToObjectives,stats.damageDealtToTurrets,...,timeline.creepsPerMinDeltas.0-10,timeline.csDiffPerMinDeltas.0-10,timeline.damageTakenDiffPerMinDeltas.0-10,timeline.damageTakenPerMinDeltas.0-10,timeline.goldPerMinDeltas.0-10,timeline.lane,timeline.participantId,timeline.role,timeline.xpDiffPerMinDeltas.0-10,timeline.xpPerMinDeltas.0-10
0,497,GOLD,1,4,3,7,9,0,995,0,...,0.2,-1.02,-25.92,395.3,160.9,NONE,1,DUO_SUPPORT,-46.96,241.1
1,20,PLATINUM,2,4,11,6,11,0,7757,0,...,0.8,-1.02,-25.92,849.9,322.3,NONE,2,DUO_SUPPORT,-46.96,351.3
2,7,GOLD,3,4,14,2,12,0,5347,3356,...,4.8,-1.02,-25.92,460.1,343.2,NONE,3,DUO_SUPPORT,-46.96,368.5
3,202,PLATINUM,4,7,4,4,10,0,1135,0,...,5.5,-1.02,-25.92,407.5,254.8,NONE,4,DUO_SUPPORT,-46.96,274.5
4,92,PLATINUM,5,14,4,2,12,0,3482,806,...,5.8,-1.02,-25.92,386.4,268.0,NONE,5,DUO,-46.96,440.1
5,236,PLATINUM,6,7,4,10,11,0,7534,5387,...,7.2,1.02,25.92,473.2,374.6,NONE,6,DUO_SUPPORT,46.96,364.8
6,412,GOLD,7,14,4,14,10,0,3871,2484,...,0.0,1.02,25.92,359.8,226.6,NONE,7,DUO_SUPPORT,46.96,260.1
7,86,UNRANKED,8,4,14,4,13,0,6792,6792,...,7.1,1.02,25.92,499.7,303.4,NONE,8,DUO_SUPPORT,46.96,462.2
8,164,UNRANKED,9,4,11,3,12,0,26022,4048,...,1.7,1.02,25.92,681.6,447.7,NONE,9,DUO_SUPPORT,46.96,403.1
9,38,UNRANKED,10,4,12,8,11,0,3712,2661,...,6.2,1.02,25.92,614.5,246.8,NONE,10,DUO_SUPPORT,46.96,420.1


Example request for a match timeline. Here, I boil down the match events to the items purchased, who purchased what, and when.

This is what I can use for finding the timing data in my join (see below).

In [14]:
# Request
test_timeline = requests.get('https://na1.api.riotgames.com/lol/match/v4/timelines/by-match/2923933470',
                            params=params)
# Normalization
test_lol_df = json_normalize(test_timeline.json(), ['frames', 'events'])

# Filter the columns I want
test_items_times = test_lol_df[['participantId', 'itemId', 'timestamp', 'type']]

# Extract the timestamp for a certain purchased item
test_items_times[(test_items_times['type'] == 'ITEM_PURCHASED') &
                 (test_items_times['participantId'] == 10) &
                 (test_items_times['itemId'] == 2033)]['timestamp'].values[-1]

761622

I need to figure out how to join the the match data with items purchased with the purchase times. The join will attach 7 purchase times onto the row with the corresponding items.

I need logic for the empty item slot (itemId = 0).

In [15]:
item_timing_col_names = ['participantId', 'item0_time', 'item1_time', 'item2_time',
                         'item3_time', 'item4_time', 'item5_time', 'item6_time']
item_timing_cols = pd.DataFrame(index=np.arange(1,11), columns=item_timing_col_names)

In [16]:
# This function will get the timestamp for a player and an item slot 0 - 6
def get_item_timestamp( participant_id, item_num ):
    col_name = 'stats.item' + str(item_num)
    
    # No item in this slot
    if (match_stats.loc[participant_id - 1, col_name] == 0):
        return np.nan
    
    # Item in the slot
    else:
        return test_items_times[(test_items_times['type'] == 'ITEM_PURCHASED') &
                         (test_items_times['participantId'] == participant_id) &
                         (test_items_times['itemId'] ==
                          match_stats[match_stats['participantId'] == participant_id][col_name].values[0]
                          )]['timestamp'].values[-1]

In [17]:
# Find all the timestamps for all players' items
for participant_id in match_stats['participantId']:
    # 7 possible final items with wards
    item_timing_cols.loc[participant_id, 'participantId'] = participant_id
    item_timing_cols.loc[participant_id, 'item0_time'] = get_item_timestamp(participant_id, 0)
    item_timing_cols.loc[participant_id, 'item1_time'] = get_item_timestamp(participant_id, 1)
    item_timing_cols.loc[participant_id, 'item2_time'] = get_item_timestamp(participant_id, 2)
    item_timing_cols.loc[participant_id, 'item3_time'] = get_item_timestamp(participant_id, 3)
    item_timing_cols.loc[participant_id, 'item4_time'] = get_item_timestamp(participant_id, 4)
    item_timing_cols.loc[participant_id, 'item5_time'] = get_item_timestamp(participant_id, 5)
    item_timing_cols.loc[participant_id, 'item6_time'] = get_item_timestamp(participant_id, 6)

This is the way I will join the match data and the item purchase data. All the way on the right, you can see the item purchase times in milliseconds.

In [18]:
match_data_with_timing_df = match_stats.join(item_timing_cols, on='participantId', lsuffix='l', rsuffix='r')
match_data_with_timing_df.head()

,championId,highestAchievedSeasonTier,participantIdl,spell1Id,spell2Id,stats.assists,stats.champLevel,stats.combatPlayerScore,stats.damageDealtToObjectives,stats.damageDealtToTurrets,...,timeline.xpDiffPerMinDeltas.0-10,timeline.xpPerMinDeltas.0-10,participantIdr,item0_time,item1_time,item2_time,item3_time,item4_time,item5_time,item6_time
0,497,GOLD,1,4,3,7,9,0,995,0,...,-46.96,241.1,1,545171,1092086,NaN,426678,NaN,NaN,830271
1,20,PLATINUM,2,4,11,6,11,0,7757,0,...,-46.96,351.3,2,617417,9046,821830,976572,982902,NaN,9937
2,7,GOLD,3,4,14,2,12,0,5347,3356,...,-46.96,368.5,3,819322,568416,168238,1003418,11125,1013153,1027529
3,202,PLATINUM,4,7,4,4,10,0,1135,0,...,-46.96,274.5,4,4987,179110,844332,1100601,583254,1000411,5548
4,92,PLATINUM,5,14,4,2,12,0,3482,806,...,-46.96,440.1,5,220964,222449,815758,220964,978267,695315,48620


In [19]:
match_stats.head()

,championId,highestAchievedSeasonTier,participantId,spell1Id,spell2Id,stats.assists,stats.champLevel,stats.combatPlayerScore,stats.damageDealtToObjectives,stats.damageDealtToTurrets,...,timeline.creepsPerMinDeltas.0-10,timeline.csDiffPerMinDeltas.0-10,timeline.damageTakenDiffPerMinDeltas.0-10,timeline.damageTakenPerMinDeltas.0-10,timeline.goldPerMinDeltas.0-10,timeline.lane,timeline.participantId,timeline.role,timeline.xpDiffPerMinDeltas.0-10,timeline.xpPerMinDeltas.0-10
0,497,GOLD,1,4,3,7,9,0,995,0,...,0.2,-1.02,-25.92,395.3,160.9,NONE,1,DUO_SUPPORT,-46.96,241.1
1,20,PLATINUM,2,4,11,6,11,0,7757,0,...,0.8,-1.02,-25.92,849.9,322.3,NONE,2,DUO_SUPPORT,-46.96,351.3
2,7,GOLD,3,4,14,2,12,0,5347,3356,...,4.8,-1.02,-25.92,460.1,343.2,NONE,3,DUO_SUPPORT,-46.96,368.5
3,202,PLATINUM,4,7,4,4,10,0,1135,0,...,5.5,-1.02,-25.92,407.5,254.8,NONE,4,DUO_SUPPORT,-46.96,274.5
4,92,PLATINUM,5,14,4,2,12,0,3482,806,...,5.8,-1.02,-25.92,386.4,268.0,NONE,5,DUO,-46.96,440.1


## Champion Table 

This imports champion basic data: name, role, championId.

I don't end up using it, but it was good practice for extracting the data and manipulating the JSON.

The data was extracted from the static API / Data Dragon.

Here is an example of the data for the champion Aatrox.

Data Dragon is currently down. I hope it comes back soon.

In [21]:
champions_request = requests.get('https://ddragonexplorer.com/cdn/8.22.1/data/en_US/champion.json')
champions_json = champions_request.json()
champions_json['data']['Aatrox']

ValueError: No JSON object could be decoded

I build the champion table here. The tricky part is that in order for the formatting of the index to work, I needed to set it after I built the rest of the table.

In [12]:
# Get index values / champion IDs
champions_idx = [str(key) for key in champions_json['data'].keys()]

# Rest of the df
champions_df = json_normalize(champions_json['data'].values())

# Set the index
champions_df.index = champions_idx
champions_df.index = champions_df.index.rename('champion_name')
champions_df.head()

,blurb,id,image.full,image.group,image.h,image.sprite,image.w,image.x,image.y,info.attack,...,stats.movespeed,stats.mp,stats.mpperlevel,stats.mpregen,stats.mpregenperlevel,stats.spellblock,stats.spellblockperlevel,tags,title,version
champion_name,,,,,,,,,,,,,,,,,,,,,
MonkeyKing,Wukong is a vastayan trickster who uses his st...,MonkeyKing,MonkeyKing.png,champion,48,champion2.png,48,96,48,8,...,345,265.84,38,8.042,0.650,32.1,1.25,"[Fighter, Tank]",the Monkey King,8.22.1
Jax,Unmatched in both his skill with unique armame...,Jax,Jax.png,champion,48,champion1.png,48,144,48,7,...,350,338.80,32,7.576,0.700,32.1,1.25,"[Fighter, Assassin]",Grandmaster at Arms,8.22.1
Kayn,A peerless practitioner of lethal shadow magic...,Kayn,Kayn.png,champion,48,champion1.png,48,192,96,10,...,340,410.00,50,11.500,0.950,32.1,1.25,"[Fighter, Assassin]",the Shadow Reaper,8.22.1
Shaco,Crafted long ago as a plaything for a lonely p...,Shaco,Shaco.png,champion,48,champion3.png,48,336,0,8,...,350,297.20,40,7.156,0.450,32.1,1.25,[Assassin],the Demon Jester,8.22.1
Warwick,Warwick is a monster who hunts the gray alleys...,Warwick,Warwick.png,champion,48,champion4.png,48,432,0,9,...,335,280.00,35,7.466,0.575,32.1,1.25,"[Fighter, Tank]",the Uncaged Wrath of Zaun,8.22.1


This is a nicer way to look at the more relevant parts of this dataframe.

In [13]:
champion_cols = ['name', 'id', 'key', 'tags', 'info.attack', 'info.defense', 'info.difficulty', 'info.magic']
champions_df_min = champions_df[champion_cols]
champions_df_min.head()

,name,id,key,tags,info.attack,info.defense,info.difficulty,info.magic
champion_name,,,,,,,,
MonkeyKing,Wukong,MonkeyKing,62,"[Fighter, Tank]",8,5,3,2
Jax,Jax,Jax,24,"[Fighter, Assassin]",7,5,5,7
Kayn,Kayn,Kayn,141,"[Fighter, Assassin]",10,6,8,1
Shaco,Shaco,Shaco,35,[Assassin],8,4,9,6
Warwick,Warwick,Warwick,19,"[Fighter, Tank]",9,5,3,3


## Item Table

This imports the item data. I use several columns from it for exploratory data analysis and to help assign item names to otherwise unknown data points.

The request gets the JSON for all purchasable items, acquired from the static API / Data Dragon.

Here is an example item: Targon's Brace.

In [14]:
items_request = requests.get('https://ddragonexplorer.com/cdn/8.22.1/data/en_US/item.json')
lol_items_json = items_request.json()
lol_items_json['data'].values()[1]

{u'colloq': u";Targon's Brace;Relic Shield;Support",
 u'description': u'<stats>+60 Health<br>+50% Base Health Regen <br>+3 Gold per 10 seconds </stats><br><br><unique>UNIQUE Passive - Spoils of War:</unique> Melee basic attacks execute minions below 200 (+40 per level) Health. Killing a minion heals the owner and the nearest allied champion for 25 Health (+2% missing health) and grants them kill Gold. 50% healing if the owner is ranged. These effects require a nearby ally. Recharges every 20 seconds. Max 3 charges.<br><br><groupLimit>Limited to 1 Starter item.</groupLimit>',
 u'effect': {u'Effect10Amount': u'0',
  u'Effect11Amount': u'5000',
  u'Effect12Amount': u'20',
  u'Effect13Amount': u'3',
  u'Effect14Amount': u'0',
  u'Effect15Amount': u'40',
  u'Effect16Amount': u'0.02',
  u'Effect17Amount': u'0.5',
  u'Effect18Amount': u'10',
  u'Effect1Amount': u'200',
  u'Effect2Amount': u'25',
  u'Effect3Amount': u'2',
  u'Effect4Amount': u'0',
  u'Effect5Amount': u'0',
  u'Effect6Amount': 

The tricky part for this dataframe was only wanting certain columns. I named them all out manually, both for their keys in the JSON and for what I wanted the dataframe columns to be called.

In [15]:
item_cols = ['name', 'description', 'consumed', 'gold.base', 'depth', 'maps.11', 'effect.Effect1Amount', 'effect.Effect2Amount',
             'effect.Effect3Amount', 'effect.Effect4Amount', 'effect.Effect5Amount','effect.Effect6Amount',
             'effect.Effect7Amount', 'effect.Effect8Amount', 'from', 'gold.purchasable', 'gold.total', 'requiredChampion',
             'specialRecipe', 'stacks', 'stats.FlatArmorMod', 'stats.FlatCritChanceMod', 'stats.FlatHPPoolMod',
             'stats.FlatHPRegenMod', 'stats.FlatMagicDamageMod', 'stats.FlatMovementSpeedMod', 'stats.FlatPhysicalDamageMod',
             'stats.FlatSpellBlockMod', 'stats.PercentAttackSpeedMod', 'stats.PercentLifeStealMod',
             'stats.PercentMovementSpeedMod', 'tags']
item_col_names = ['name', 'description', 'consumed', 'base_gold', 'depth', 'sr', 'effect1amount', 'effect2amount',
                 'effect3amount', 'effect4amount', 'effect5amount', 'effect6amount', 'effect7amount', 'effect8amount', 'from',
                 'gold_purchasable', 'total_gold', 'req_champion', 'special_recipe', 'stacks', 'flat_armor_mod',
                 'flat_crit_chance_mod', 'flat_hp_pool_mod', 'flat_hp_regen_mod', 'flat_magic_dmg_mod', 'flat_ms_mod',
                 'flat_phys_dmg_mod', 'flat_spellblock_mod', 'flat_pct_atk_speed_mod', 'pct_lifesteal_mod', 'pct_movespeed_mod',
                  'tags']
lol_items_df = json_normalize(data=lol_items_json['data'].values())[item_cols]
lol_items_df.columns = item_col_names
lol_items_df.head()

,name,description,consumed,base_gold,depth,sr,effect1amount,effect2amount,effect3amount,effect4amount,...,flat_hp_pool_mod,flat_hp_regen_mod,flat_magic_dmg_mod,flat_ms_mod,flat_phys_dmg_mod,flat_spellblock_mod,flat_pct_atk_speed_mod,pct_lifesteal_mod,pct_movespeed_mod,tags
0,Skirmisher's Sabre,<groupLimit>Limited to 1 Gold Income or Jungle...,NaN,300,2,True,80,30,5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[LifeSteal, ManaRegen, OnHit, Jungle]"
1,Heart of Targon,<stats>+60 Health<br>+50% Base Health Regen <b...,NaN,400,NaN,False,200,25,2,0,...,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Health, HealthRegen, Aura, GoldPer, Lane]"
2,Philosopher's Medallion,<stats>+10% Cooldown Reduction<br>+50% Base He...,NaN,450,NaN,False,40,10,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[HealthRegen, ManaRegen, GoldPer, CooldownRedu..."
3,Salvation,<stats><font color='#FFFFFF'>+300 Health</font...,NaN,500,4,True,0.1,10,20,0.1,...,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Health, HealthRegen, ManaRegen, CooldownReduc..."
4,Ghost Poro,<subtitleLeft><font color='#FFFFFF'>(Trinket)<...,True,0,NaN,False,240,3.5,42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Vision, Trinket, Active]"


As with the champion dataframe, I need to set the set the index separately.

In [16]:
lol_items_df_idx = [str(key) for key in lol_items_json['data'].keys()]
lol_items_df.index = lol_items_df_idx
lol_items_df.index = lol_items_df.index.rename('item_id')

In [17]:
# Filter for only Summoner's Rift items
lol_items_df = lol_items_df[lol_items_df['sr'] == True].fillna(0)
lol_items_df = lol_items_df.sort_index()

# Here is what the data looks like now
lol_items_df.head()

,name,description,consumed,base_gold,depth,sr,effect1amount,effect2amount,effect3amount,effect4amount,...,flat_hp_pool_mod,flat_hp_regen_mod,flat_magic_dmg_mod,flat_ms_mod,flat_phys_dmg_mod,flat_spellblock_mod,flat_pct_atk_speed_mod,pct_lifesteal_mod,pct_movespeed_mod,tags
item_id,,,,,,,,,,,,,,,,,,,,,
1001,Boots of Speed,<groupLimit>Limited to 1 pair of boots.</group...,0,300,0,True,0,0,0,0,...,0,0,0,25,0,0,0,0,0,[Boots]
1004,Faerie Charm,<stats><mana>+25% Base Mana Regen </mana></stats>,0,125,0,True,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[ManaRegen]
1006,Rejuvenation Bead,<stats>+50% Base Health Regen </stats>,0,150,0,True,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[HealthRegen]
1011,Giant's Belt,<stats>+380 Health</stats>,0,600,2,True,0,0,0,0,...,380,0,0,0,0,0,0,0,0,[Health]
1026,Blasting Wand,<stats>+40 Ability Power</stats>,0,850,0,True,0,0,0,0,...,0,0,40,0,0,0,0,0,0,[SpellDamage]


This is what an item's data looks like from the dataframe.

In [25]:
lol_items_df.loc['2010', :]

name                                      Total Biscuit of Everlasting Will
description               <consumable>Click to Consume:</consumable> Res...
consumed                                                               True
base_gold                                                                75
depth                                                                     0
sr                                                                     True
effect1amount                                                            15
effect2amount                                                             0
effect3amount                                                             0
effect4amount                                                             0
effect5amount                                                             0
effect6amount                                                             0
effect7amount                                                             0
effect8amoun

Clean the item table. I think I can just fill all NaNs with 0.

## Match Data Acquisition

Need random players in Platinum and Diamond leagues, and random ranked games from their history. Then get all that match data, and boil it down with the same logic that I used above.

To start, I need to find the Platinum and Diamond leagues.

League IDs from plat and diamond leagues. I found a csv file on a thread (https://discussion.developer.riotgames.com/questions/5696/how-to-get-all-leagueid.html) with a player-made database of league IDs by region (http://canisback.com/leagueId/ ). I downloaded 'league_na1.csv' and renamed it to 'league_ids_na1.csv'

In [16]:
league_id_df = pd.read_csv('../data/league_ids_na1.csv')
league_id_df.head()

,leagueId,tier,number
0,000115d0-45ad-11e8-a09c-c81f66dbb56c,GOLD,200
1,000220d0-8ac5-11e8-ae23-c81f66cf2333,SILVER,200
2,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM,105
3,000707c0-35df-11e8-a82d-c81f66cf2333,GOLD,200
4,0010adb0-04a8-11e8-9e8c-c81f66cf135e,SILVER,200


In [17]:
league_id_df.groupby('tier').size()

tier
BRONZE        1438
CHALLENGER       1
DIAMOND        222
GOLD          2926
MASTER           1
PLATINUM       881
SILVER        4546
dtype: int64

Keep Diamond and Platinum league IDs

In [18]:
diamond_plat_ids = league_id_df[(league_id_df['tier'] == 'PLATINUM') | (league_id_df['tier'] == 'DIAMOND')]
len(diamond_plat_ids)

1103

In [19]:
diamond_plat_ids.head()

,leagueId,tier,number
2,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM,105
6,001eef10-0467-11e8-a61f-c81f66cf2333,DIAMOND,200
40,00d11cd0-d775-11e8-8841-c81f66cf2333,PLATINUM,112
41,00d1b910-d775-11e8-8841-c81f66cf2333,PLATINUM,167
42,00d1b911-d775-11e8-8841-c81f66cf2333,PLATINUM,104


Build a list of players from the selected league IDs. This takes a couple minutes.

In [83]:
players_list_per_league = []
for leagueId in diamond_plat_ids['leagueId']:
    league_players = requests.get('https://na1.api.riotgames.com/lol/league/v4/leagues/' + leagueId, params=params)
    players_list_per_league.append(json_normalize(diamond_plat_players.json(), record_path='entries', meta=['leagueId', 'tier']))

In [84]:
len(players_list_per_league)

1103

In [103]:
players_per_league_df = pd.concat(players_list_per_league)
players_per_league_df = players_per_league_df.reset_index(drop=True)
len(players_per_league_df)

118021

Wow! 118k players.

In [105]:
players_per_league_df.head(2)

,freshBlood,hotStreak,inactive,leaguePoints,losses,miniSeries,rank,summonerId,summonerName,veteran,wins,leagueId,tier
0,False,False,False,7,156,NaN,V,XeLmUgisomeVZV5sxnPZ_i_rvy3tY6UzTuwWMVF_6oqDXQ4,JohnLaeE,False,169,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
1,False,False,False,87,90,NaN,IV,9tt-xsc6lJWaklLQb3cZ67JjKT9as9Twe2lhC6zLZ_ZbR54,ALEDSO,False,134,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM


Save the table of players so that I don't have to do this again.

In [98]:
players_per_league_df.to_csv('../data/summonerIds.csv', encoding='utf-8')

Read table of players.

In [20]:
players_per_league_df = pd.read_csv('../data/summonerIds.csv', encoding='utf-8', index_col=0)
players_per_league_df.head()

,freshBlood,hotStreak,inactive,leaguePoints,losses,miniSeries,rank,summonerId,summonerName,veteran,wins,leagueId,tier
0,False,False,False,7,156,NaN,V,XeLmUgisomeVZV5sxnPZ_i_rvy3tY6UzTuwWMVF_6oqDXQ4,JohnLaeE,False,169,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
1,False,False,False,87,90,NaN,IV,9tt-xsc6lJWaklLQb3cZ67JjKT9as9Twe2lhC6zLZ_ZbR54,ALEDSO,False,134,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
2,False,True,False,0,119,NaN,V,ArNNtm1KFNm6f0OVQAY_hHUBeoqXXYrRWloV337NZPoKjtc,Cute School Girl,False,135,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
3,False,False,False,34,47,NaN,V,1904JnVQBXOoScRAwJ0_suX0YInL7kkk4dgZ_q3-XobUqIs,iHardScop3ftw,False,59,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
4,False,False,False,2,32,NaN,V,5oDyD3ShWA_nuiNWGLv9DjD6R1pYvT5W19aqFGySRp2QzQw,TieuTieu,False,51,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM


Select a random subset of players

In [21]:
players_sample = players_per_league_df.sample(n=1000, axis='index')
players_sample.head()

,freshBlood,hotStreak,inactive,leaguePoints,losses,miniSeries,rank,summonerId,summonerName,veteran,wins,leagueId,tier
80474,False,False,False,17,189,NaN,V,oTj32GWP1eKnScdkiY4JOE-ae_P8_tn8Yw3sdyOVBxxk5Ts,RedShowbiz,False,180,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
17929,False,False,False,0,19,NaN,V,QemJEsvrcRlshgRKqeOye-nYdkOruPu8eyKIutboo5zj,2needles,False,29,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
18681,False,False,False,0,55,NaN,V,LGexYmt0TgiRPUDZfxuONHiPHjwpWve-O8KQwgDuU9Is0c4,secret house,False,59,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
62432,False,False,False,20,64,NaN,V,6yuSAEsz1st_5y5irj-TPGkdf2VQOyn0QlRbGb040f8-zYo,Xiqe,False,63,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM
48086,False,False,False,0,102,NaN,V,IzJh2jHnpR3yE8AnralfaBecT6tzeBi6eOi6wJ1e5baPGow,BigJiggleyBooBs,False,139,000305f0-dc2c-11e8-b3fa-c81f66dbb56c,PLATINUM


In [22]:
len(players_sample)

1000

From summonerID, get accountID. Started 1000 requests at 12:58, finished in maybe 5 min.

In [23]:
account_id_list = []
for summonerId in players_sample['summonerId']:
    account = requests.get('https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + summonerId, params=params)
    account_id_list.append(json_normalize(account.json()))

In [24]:
len(account_id_list)

1000

In [25]:
account_id_df = pd.concat(account_id_list)
account_id_df.head()

,accountId,id,name,profileIconId,puuid,revisionDate,status.message,status.status_code,summonerLevel
0,jcca11rEECVt92_fO-wROSTXjfe8Hg4QsFYLawdcz51Sazo,oTj32GWP1eKnScdkiY4JOE-ae_P8_tn8Yw3sdyOVBxxk5Ts,RedShowbiz,0,1Tq_gJJqWlD3v9ZIOepd3FOLpPqWHwnZ9wtyzNBSHLQA0f...,1.543955e+12,NaN,NaN,61
0,trDelhyIAQafBMCRfNN1GES4ZQoosvnAy1P1acEVziw,QemJEsvrcRlshgRKqeOye-nYdkOruPu8eyKIutboo5zj,2needles,8,gxIfoAubGK4gZJ256bDo1A_alBthIF-kSMRlRzeCR5JOaa...,1.541031e+12,NaN,NaN,35
0,qF_ruKGLkMWEMNFEZ_zIt-1X9mJJ3v8NUNRxcyBkReccFRM,LGexYmt0TgiRPUDZfxuONHiPHjwpWve-O8KQwgDuU9Is0c4,secret house,3175,bgxEYrLbTXn8hz11Qm56qDqGhG4ngA5o0M0BVjLN2_3GHX...,1.540939e+12,NaN,NaN,52
0,WmjmGrn29WyhsLpkIkAqfnDDJtbA5hbrJt1qZAIiiWaBzw,6yuSAEsz1st_5y5irj-TPGkdf2VQOyn0QlRbGb040f8-zYo,Xiqe,3795,6liYKrxX-1J1gmsqpLPuR8nSw08B05MrTitAg-LF6H04fe...,1.543815e+12,NaN,NaN,93
0,Cq0YZy3jXTMpp8j8AccpkjSLOtn5r8go5M1sxqAR6TKFtso,IzJh2jHnpR3yE8AnralfaBecT6tzeBi6eOi6wJ1e5baPGow,BigJiggleyBooBs,7,56TM7Y9fOoMYb-BJFMIzwH8J5Hc-PzqFg7z6s2I1NbCCtw...,1.543204e+12,NaN,NaN,87


In [44]:
len(account_id_df)

1000

Get match history for each player.

In [183]:
match_history_list = []
for accountId in account_id_df['accountId']:
    match_history = requests.get('https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + str(accountId) + '?queue=420',
                                params=params)
    if 'matches' in match_history.json():
        match_history_list.append(json_normalize(match_history.json()['matches']))
    time.sleep(1.2)

In [184]:
len(match_history_list)

196

In [185]:
match_history_df = pd.concat(match_history_list)
len(match_history_df)

18091

Make a set of the gameIds, then get match data from gameId. gameId is long, need to convert to int and then to str

In [248]:
match_history_df.head()

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,126,2924316053,MID,NA1,420,SOLO,11,1543952595125
1,126,2924333379,NONE,NA1,420,DUO_SUPPORT,11,1543951246581
2,126,2924193158,MID,NA1,420,SOLO,11,1543908576060
3,126,2924177627,MID,NA1,420,DUO,11,1543906450796
4,126,2924029423,MID,NA1,420,SOLO,11,1543893366771


Take a random sample of 1000 games from the match dataframe. Make sure the game IDs are unique, in case some players in my random sample of players faced each other.

In [ ]:
game_ids = match_history_df.sample(n=1000, axis='index')['gameId'].unique()

Make list of dataframes for each match, then turn into one big dataframe? Rate limits mean that fetching ~900 games should take 40 min

In [41]:
game_ids[0:10]

array([2855810690, 2894635369, 2792282086, 2842012227, 2889141116,
       2907087506, 2892832264, 2893954890, 2898065427, 2907077614], dtype=int64)

In [64]:
def get_item_timestamp( match_df, timing_df, participant_id, item_num ):
    
    col_name = 'stats.item' + str(item_num)
    blacklisted_item_ids = [2424, 3340, 2421, 3042, 2422, 3040, 2403, 3513, 2010]
    item_id = match_df[match_df['participantId'] == participant_id][col_name].values[0]
        
    if (match_df.loc[participant_id - 1, col_name] == 0) or (item_id in blacklisted_item_ids):
        return np.nan
    else:
        timestamps = timing_df[(timing_df['type'] == 'ITEM_PURCHASED') &
                         (timing_df['participantId'] == participant_id) &
                         (timing_df['itemId'] ==
                          match_df[match_df['participantId'] == participant_id][col_name].values[0]
                          )]['timestamp']
        if len(timestamps) == 0:
            return np.nan
        else:
            return timestamps.values[-1]

Here is the big request loop.

For each game ID, I get the match data, normalized properly, and join the timing data to it.

Also need to be sure to not violate my API key's request rate of 100 requests per 2 minutes.

In [66]:
match_data_list = []
item_timing_col_names = ['participantId', 'item0_time', 'item1_time', 'item2_time',
                         'item3_time', 'item4_time', 'item5_time', 'item6_time']
match_data
for game_id in game_ids:
    # Requests
    match_data = requests.get('https://na1.api.riotgames.com/lol/match/v4/matches/' + str(game_id), params=params)
    match_timeline = requests.get('https://na1.api.riotgames.com/lol/match/v4/timelines/by-match/' + str(game_id), params=params)
    
    # Dataframes
    if ('status' in match_timeline.json() and match_timeline.json()['status']['status_code'] == 404):
        continue
    
    timeline_df = json_normalize(match_timeline.json(), ['frames', 'events'])
    timeline_df = timeline_df[['participantId', 'itemId', 'timestamp', 'type']]
    
    match_data_df = json_normalize(match_data.json()['participants']).sort_values('participantId')
    
    item_timing_cols = pd.DataFrame(index=np.arange(1,11), columns=item_timing_col_names)
    
    # Fill in item_timing_cols
    for participant_id in match_data_df['participantId']:
    # 7 possible final items with wards
        item_timing_cols.loc[participant_id, 'participantId'] = participant_id
        item_timing_cols.loc[participant_id, 'item0_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 0)
        item_timing_cols.loc[participant_id, 'item1_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 1)
        item_timing_cols.loc[participant_id, 'item2_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 2)
        item_timing_cols.loc[participant_id, 'item3_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 3)
        item_timing_cols.loc[participant_id, 'item4_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 4)
        item_timing_cols.loc[participant_id, 'item5_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 5)
        item_timing_cols.loc[participant_id, 'item6_time'] = get_item_timestamp(match_data_df, timeline_df, participant_id, 6)
    
    match_data_with_timing_df = match_data_df.join(item_timing_cols, on='participantId', lsuffix='_l', rsuffix='_r')
    
    match_data_list.append(match_data_with_timing_df)
    time.sleep(2.5)

In [67]:
len(match_data_list)

925

In [68]:
matches_df = pd.concat(match_data_list)

Here is the final dataframe!

In [69]:
matches_df.head(10)

,championId,highestAchievedSeasonTier,item0_time,item1_time,item2_time,item3_time,item4_time,item5_time,item6_time,masteries,...,timeline.participantId,timeline.role,timeline.xpDiffPerMinDeltas.0-10,timeline.xpDiffPerMinDeltas.10-20,timeline.xpDiffPerMinDeltas.20-30,timeline.xpDiffPerMinDeltas.30-end,timeline.xpPerMinDeltas.0-10,timeline.xpPerMinDeltas.10-20,timeline.xpPerMinDeltas.20-30,timeline.xpPerMinDeltas.30-end
0,103,SILVER,1442408,1317001,1097684,NaN,1955521,1859933,NaN,NaN,...,1,DUO_CARRY,-19.3,-24.55,-50.25,NaN,332.5,458.8,424.8,NaN
1,267,SILVER,1145366,1584846,566750,1916517,1956258,1956258,NaN,NaN,...,2,DUO_SUPPORT,-19.3,-24.55,-50.25,NaN,306.0,294.8,424.2,NaN
2,24,UNRANKED,704373,1326581,959447,1989154,NaN,NaN,NaN,NaN,...,3,NONE,-43.7,-79.20,-2.30,NaN,285.1,459.3,558.5,NaN
3,61,PLATINUM,1170289,805550,1953211,1656055,1168408,1959599,NaN,NaN,...,4,SOLO,13.0,-131.10,60.50,NaN,490.0,434.0,474.4,NaN
4,41,GOLD,1592110,644571,1142726,1834880,NaN,NaN,NaN,NaN,...,5,SOLO,-18.2,-190.30,69.40,NaN,411.9,428.6,511.5,NaN
5,80,UNRANKED,15125,763991,1318684,1185242,1646738,1777846,NaN,NaN,...,6,SOLO,18.2,190.30,-69.40,NaN,430.1,618.9,442.1,NaN
6,51,GOLD,1840722,604770,1507416,788646,1158770,1651397,NaN,NaN,...,7,DUO_CARRY,19.3,24.55,50.25,NaN,392.3,503.8,623.2,NaN
7,99,UNRANKED,939744,554210,1239105,1486997,1885613,NaN,NaN,NaN,...,8,SOLO,-13.0,131.10,-60.50,NaN,477.0,565.1,413.9,NaN
8,111,GOLD,554507,1443795,1905387,789306,1907371,1907371,1240392,NaN,...,9,DUO_SUPPORT,19.3,24.55,50.25,NaN,284.8,298.9,326.3,NaN
9,64,GOLD,1903638,977667,1623427,571074,1347487,1907239,NaN,NaN,...,10,NONE,43.7,79.20,2.30,NaN,328.8,538.5,560.8,NaN


In [70]:
matches_df.to_csv('../data/match_data_2.csv', encoding='utf-8')

In [71]:
matches_df = pd.read_csv('../data/match_data_2.csv', encoding='utf-8', index_col=0)

In [72]:
matches_df.head()

,championId,highestAchievedSeasonTier,item0_time,item1_time,item2_time,item3_time,item4_time,item5_time,item6_time,masteries,...,timeline.participantId,timeline.role,timeline.xpDiffPerMinDeltas.0-10,timeline.xpDiffPerMinDeltas.10-20,timeline.xpDiffPerMinDeltas.20-30,timeline.xpDiffPerMinDeltas.30-end,timeline.xpPerMinDeltas.0-10,timeline.xpPerMinDeltas.10-20,timeline.xpPerMinDeltas.20-30,timeline.xpPerMinDeltas.30-end
0,103,SILVER,1442408,1317001,1097684,NaN,1955521,1859933,NaN,NaN,...,1,DUO_CARRY,-19.3,-24.55,-50.25,NaN,332.5,458.8,424.8,NaN
1,267,SILVER,1145366,1584846,566750,1916517,1956258,1956258,NaN,NaN,...,2,DUO_SUPPORT,-19.3,-24.55,-50.25,NaN,306.0,294.8,424.2,NaN
2,24,UNRANKED,704373,1326581,959447,1989154,NaN,NaN,NaN,NaN,...,3,NONE,-43.7,-79.20,-2.30,NaN,285.1,459.3,558.5,NaN
3,61,PLATINUM,1170289,805550,1953211,1656055,1168408,1959599,NaN,NaN,...,4,SOLO,13.0,-131.10,60.50,NaN,490.0,434.0,474.4,NaN
4,41,GOLD,1592110,644571,1142726,1834880,NaN,NaN,NaN,NaN,...,5,SOLO,-18.2,-190.30,69.40,NaN,411.9,428.6,511.5,NaN


In [73]:
len(matches_df)

9250